In [ ]:
'''
#To-do: 
魚の個体数に応じて以下の5箇所を変更する。
1.読み込みたいファイルのパスの変更. 
2.読み込みたい列番号の変更. 
3.関数do_tpmにおける以下の箇所を変更
4.bigPhi計算のための必要要素を変更
5.for文に出てくる以下を変更

~以下はその具体的説明~

遷移確率行列を出力するcell(2つ目のcell)において、
まずdataを取得する部分で
1.読み込みたいファイルのパスの変更. 
2.読み込みたい列番号の変更. 
次に後半に出てくるが、
3.関数do_tpmにおける以下の箇所を変更

>bin_str = format(j, '05b') #＜注意＞lによって変えないといけない

'05b'はl=5匹の場合なので、l=num,when 0<num<10, numは自然数の場合は'0numb'と変える


calculate bigPhiのcell(3つめのcell)において、
4.bigPhi計算のための必要要素を変更
>labels = ('A','B','C','D','E')
l=5 # 念の為．上と同じ値ではある．
ll=5400 # 念の為．上と同じ値ではある．

を変更する。例では、l=5匹の場合であるので、labelsにはA~Eの5つのラベルがセットされている。llはデータの長さをRawData_to_CalculationDataのプログラムで5400合計単位時間分にそろえているので基本的には変更の必要はない。
また、
5.for文に出てくる以下を変更
>innter_list.append(tuple(map(int, (format(i, '05b'))))) #lの値によって変える必要あり

の部分も3.の修正と同様に変更する
'''

In [1]:
####################################################
#極性ネマチック状態を考慮した統合情報量と相互情報量の算出
#author:wakabayashi
#condition：5匹，極性ネマチック式のε=0.3のとき
#data：2019/12/05
#attention:データや条件は目的に応じて変えましょう．
####################################################

import numpy as np
import math

data = np.loadtxt("/home/qcmp/workspace/undergraduate_project/position_data/moving_average_12_5_5.csv",       # 読み込みたいファイルのパス
                  delimiter=" ",    # ファイルの区切り文字
                  skiprows=0,       # 先頭の何行を無視するか（指定した行数までは読み込まない）
                  usecols=(0,1,2,3,4,5,6,7,8,9)) # 読み込みたい列番号

l = int((data.shape[1])/2) #魚の個体数を表す．２で割っているのは，各個体につきx,y座標の２つのデータが存在するから．
ll = int(data.shape[0]) # 時系列のデータの長さ

######################################################

def distance():
    a = np.zeros((ll, l, l))
    #a = np.zeros(int(len(x)/2),int(len(x)/2)) ではない
    for h in range (ll):
        for i in range(l):
            for j in range(l):
                a[h][i][j] = math.sqrt(((data[h][2*i] - data[h][2*j])**2)+(data[h][2*i+1] - data[h][2*j+1])**2) 
    return a


def velocity():
    dt = 1/6 #魚の知覚時間感覚に近い値を採用．(30fpsで撮影した動画の５フレームの移動平均をとるので，実際の動画のサンプリング間隔も1/6とした．)

    vdata = np.zeros([ll-1, l*2])

    for u in range(ll-1):
        vdata[u][:] = (data[u+1][:] - data[u][:])/dt
        
    return vdata


def normalized_direction(vdata):
    normal_dir = np.zeros([ll-1, l*2])
    for p in range(ll-1):
        for q in range(l):
            normal_dir[p][2*q] =  vdata[p][2*q] / (np.sqrt(vdata[p][2*q]**2 + vdata[p][2*q+1]**2)+1e-15) #+1e-15足して0で割るのを防ぐ
            normal_dir[p][2*q+1] = vdata[p][2*q+1] / (np.sqrt(vdata[p][2*q]**2 + vdata[p][2*q+1]**2)+1e-15)
    return normal_dir

def distance_vector():
    dis = np.zeros((ll, l, l*2))
    for h in range (ll):
        for i in range(l):
            for j in range(l):
                dis[h][i][2*j] = data[h][2*i] - data[h][2*j]
                dis[h][i][2*j+1] = data[h][2*i+1] - data[h][2*j+1]

    return dis



def inner1(m, n):
    inn1 = np.zeros((ll-1, l, l))
    for h in range (ll-1):
        for i in range(l):
            for j in range(l):
                inn1[h][i][j] = m[h][2*i] * n[h][2*j] + m[h][2*i+1] * n[h][2*j+1]
    return inn1

def inner2(m, n):
    inn2 = np.zeros((ll-1, l, l))
    h = i = j = k = 0
    #print(m.shape)
    #n = np.transpose(n, (0,2,1)) 
    for h in range (ll-1):
        for i in range(l):
            for j in range(l):
                    inn2[h][i][j] = m[h][i][2*j] * n[h][i][2*j] + m[h][i][2*j+1] * n[h][i][2*j+1]
    return inn2

def inner3(m, n):
    inn3 = np.zeros((ll-1, l, int(l/2)))
    h = i = j = k = 0
    for h in range (ll-1):
        for i in range(l):
            for j in range(int(l/2)):
                for k in range(int(l/2)): 
                    inn3[h][i][j] = m[h][i][2*j] * n[h][i][2*k] + m[h][i][2*j+1] * n[h][i][2*k+1]
    inn3 = inn3.reshape(ll-1, l)
    return inn3

def potential():#極性ネマチック状態の算出
    mul = np.zeros((ll-1, l, l))
    g = np.zeros((ll-1, l, l))
    ep = 0.3 # 極性ネマチック式のεに相当 
    for h in range (ll-1):
        for i in range(l):
            for j in range(l):
                if (ad[h][i][j] == 0):
                    g[h][i][j] = 0
                else:
                    g =  -1 / (ad[h][i][j]**3 ) * (inner1(normal_dir, normal_dir) - 3 * ep / (ad[h][i][j]**2) * np.multiply(inner2(normal_dird, disd),inner2(normal_dird, disd).transpose(0,2,1)))　#論文パワポはこの式の頭のマイナスを忘れているかも．
    return g

def average(a):
    ave = a/(sum(a))
    return ave

def min_max(x, axis=None):
    x_min = x.min(axis=axis, keepdims=True)
    x_max = x.max(axis=axis, keepdims=True)
    return (x - x_min) / (x_max - x_min)


def do_tpm(z3):
    tpm = np.zeros((2**l, l))
    z4 = np.zeros([ll-1,l])

    for p in range(ll-1):
        for q in range(l):
           z4[p][q] = z3[p][l-q-1]#列ベクトルを逆順にする
    
    temp = np.zeros((2**l, l)) + 1
    tp = np.zeros((2**l)) + 1
    tpm = np.zeros((2**l, l)) 
    tpp = np.zeros((2**l)) 
    for i in range(ll-2):
        for j in range(2**l):
            bin_str = format(j, '05b') #＜注意＞lによって変えないといけない
            for k in range(l):
                if int(bin_str[k]): #文字列の右からk番目が1のとき
                    temp[j][:] *= z4[i][k] #時系列データの表す確率をかける
                elif z4[i][k]>=0:
                    temp[j][:] *= (1-z4[i][k])/((ll-1)*l)
                elif z4[i][k]<0:
                    temp[j][:] *= -(1+z4[i][k])/((ll-1)*l)
                tp[j]*=temp[j][k] #各状態の現れる確率を計算しておくtp[j]=temp[j][:] #各状態の現れる確率をk計算しておく
                temp[j][k] *= z4[i+1][k]
            tpm[j][:] += temp[j][:]
            tpp[j] += tp[j] #引数指定しないと変なふうにたされる。要確認
            temp = np.zeros((2**l, l)) + 1
            tp = np.zeros((2**l)) + 1
    return tpp, tpm #averageとるとconditional independenceでなくるるけど，when not average, fai got 0.

a = distance()
ad = np.delete(a, ll-1, 0) #shape(ll-1, l, l)

dis = distance_vector() 
disd = np.delete(dis, ll-1, 0) #shape(ll-1, l, l*2)

vdata = velocity()
normal_dir = normalized_direction(vdata) #shape(ll-1, l*2)
normal_dird = np.tile(normal_dir.reshape(ll-1,1,l*2),(1,l,1))

g = potential()
z3 = np.sum(g, axis=2)

tpp, tpm = do_tpm(z3)
ave_tpm = min_max(tpm)
ave_tpp = average(tpp)
print(ave_tpm)
print(ave_tpp)

[[8.60842808e-03 0.00000000e+00 1.74883360e-02 1.78414867e-02
  2.49976453e-02]
 [4.33032420e-02 9.56969694e-03 4.99347888e-02 4.36242760e-02
  7.26301918e-02]
 [4.93265343e-02 6.56428490e-04 5.76640157e-02 4.64895506e-02
  6.18630198e-02]
 [1.14965558e-01 2.29647265e-02 1.18534421e-01 1.03664069e-01
  1.39182591e-01]
 [5.09325422e-02 1.63872845e-02 5.11969417e-02 4.73196083e-02
  5.84495187e-02]
 [1.14932841e-01 4.82452786e-02 1.16919434e-01 1.06548797e-01
  1.37100194e-01]
 [1.18772162e-01 2.22592801e-02 1.19125990e-01 1.09678858e-01
  1.24902303e-01]
 [2.41583045e-01 7.57037364e-02 2.37979039e-01 2.26431555e-01
  2.61418534e-01]
 [4.58456782e-02 1.13833371e-02 4.91884379e-02 4.20424141e-02
  6.19446696e-02]
 [1.03242528e-01 2.64052754e-02 1.06421570e-01 9.21007887e-02
  1.55414877e-01]
 [1.09075023e-01 1.08870783e-02 1.05492088e-01 9.75986088e-02
  1.28040223e-01]
 [2.19483413e-01 5.09353241e-02 2.11650880e-01 2.01036251e-01
  2.71756340e-01]
 [1.09794176e-01 4.58786883e-02 1.077852

In [3]:
# calculate bigPhi
!pip install pyphi
import pyphi
pyphi.conf.PyphiConfig.CUT_ONE_APPROXIMATION = True
labels = ('A','B','C','D','E')
l=5 # 念の為．上と同じ値ではある．
ll=5400 # 念の為．上と同じ値ではある．
tpm = ave_tpm
tpp = ave_tpp
c = np.zeros((2**l))
innter_list = []
for i in range(2**l):
    innter_list.append(tuple(map(int, (format(i, '05b'))))) #lの値によって変える必要あり
for t in range(2**l):
    state = innter_list[2**l-t-1]
    network = pyphi.Network(tpm, node_labels = labels)
    subsystem = pyphi.Subsystem(network, state)
    c[t] = pyphi.compute.phi(subsystem)
print(c)
bigphi = np.dot(c, tpp)
print(bigphi)

You should consider upgrading via the 'pip install --upgrade pip' command.


[6.956652 2.378029 1.071643 0.431824 2.483585 0.599763 0.427709 0.368813
 1.710347 0.771062 0.455346 0.308738 0.772644 0.406749 0.333277 0.373385
 2.757875 0.853058 0.384782 0.350592 0.733066 0.493142 0.367538 0.488163
 0.7672   0.472343 0.287295 0.350703 0.386478 0.477212 0.375622 0.487761]
0.4102997033074099


In [3]:
just copy the results
[7.006405 2.585445 1.301484 0.444643 2.449336 0.772065 0.420311 0.408775
 1.669805 0.747394 0.519234 0.319268 0.764619 0.497854 0.362326 0.442671
 2.747098 0.698885 0.427349 0.323194 0.635525 0.486146 0.359873 0.481006
 0.754558 0.392865 0.303858 0.33097  0.374652 0.467761 0.387971 0.492884]
0.41838732616717317

SyntaxError: invalid syntax (<ipython-input-3-c14de9b3ad65>, line 1)

In [4]:
# calculate mutual information MI
#!pip install sklearn
from sklearn.metrics import mutual_info_score# MIの計算

mi = np.zeros((ll-1))
for t in range(ll-2):
    X = z3[t][:]
    Y = z3[t+1][:]

    # 相互情報量の計算
    mi[t]= mutual_info_score(X, Y)
    #print(mi[t])

    # 結果の出力
print(np.mean(mi))

1.6073424320784275


In [ ]:
just copy the results
1.6073424320784275